# Reddit 

Documentation: https://praw.readthedocs.io/en/stable/getting_started/quick_start.html

In [1]:
# Install the required packages

import sys 
from IPython.display import clear_output

!pip install requests
!pip install pandas
!pip install praw==7.7.1 


Looking in indexes: https://pypi.org/simple, https://private.artifactory.foc.zone/artifactory/pypi-local,https://private.artifactory.foc.zone/artifactory/api/pypi/pypi-remote/simple,https://private.artifactory.foc.zone/artifactory/api/pypi/rdf-pypi-virtual/simple
You should consider upgrading via the '/Users/ASerban/Desktop/MADS/reddit_social_media/env/bin/python3 -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://private.artifactory.foc.zone/artifactory/pypi-local,https://private.artifactory.foc.zone/artifactory/api/pypi/pypi-remote/simple,https://private.artifactory.foc.zone/artifactory/api/pypi/rdf-pypi-virtual/simple
You should consider upgrading via the '/Users/ASerban/Desktop/MADS/reddit_social_media/env/bin/python3 -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://private.artifactory.foc.zone/artifactory/pypi-local,https://private.artifactory.foc.zone/artifactory/api/pypi/pypi-remote/simple,https:

In [2]:
import json #needed to translate JSON data
import requests #needed to perform HTTP GET and POST requests
import pandas as pd
import pprint #allows us to print more readable JSON data
from datetime import datetime 
import time 

#We will use the PRAW library to access Reddit data, version 7.7.1
try:
    import praw
except:
    !pip install praw==7.7.1 #this will install the version of PRAW that we need if PRAW does not already exist
    import praw


print("praw version:",praw.__version__) #it can be helpful to confirm the version we're using for our project

praw version: 7.7.1


In [7]:
import json

# Function to load credentials from a JSON file
def load_credentials(file_path):
    with open(file_path, 'r') as file:
        credentials = json.load(file)
    return credentials

# Path to the credentials file
file_path = 'credentials.json'

# Load the credentials
credentials = load_credentials(file_path)

# Assigning the credentials to variables
REDDIT_USERNAME = credentials['REDDIT_USERNAME']
REDDIT_PASSWORD = credentials['REDDIT_PASSWORD']
APP_ID = credentials['APP_ID']
APP_SECRET = credentials['APP_SECRET']
APP_NAME = credentials['APP_NAME']

In [ ]:
#Generate your reddit instance
reddit = praw.Reddit(
    client_id=APP_ID,
    client_secret=APP_SECRET,
    user_agent=APP_NAME,
    username=REDDIT_USERNAME, 
    password=REDDIT_PASSWORD,
    check_for_async=False # This additional parameter supresses a warning about "Asynchronous PRAW"
)

In [12]:
print(reddit.user.me())

RequestException: error with request HTTPSConnectionPool(host='www.reddit.com', port=443): Max retries exceeded with url: /api/v1/access_token (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1129)')))

In [13]:
def create_submission_df(subredit_topic="FirstTimeHomeBuyer", n_posts=5, search_query='Rocket'): 
    subreddit = reddit.subreddit(subredit_topic)
    submissions = subreddit.search(search_query, limit=n_posts)

    # submissions = subreddit.hot(limit=n_posts)
    data = {
        'submission_id': [],
        'subredit_topic': [],
        'search_query': [],
        'title': [],
        'text': [],
        'score': [],
        'num_comments': [],
        'username': [],
        'created_at': [],
        # Added 
        'comment_dict': []
    }
    for submission in submissions:
        data['submission_id'].append(submission.id)
        data['subredit_topic'].append(subredit_topic)
        data['search_query'].append(search_query)
        data['title'].append(submission.title)
        data['text'].append(submission.selftext)
        data['score'].append(submission.score)
        data['num_comments'].append(submission.num_comments)
        data['username'].append(submission.author.name if submission.author else 'Deleted')
        data['created_at'].append(submission.created_utc)

        # added to get a dict of comments
        comment_dict = {}
        for i, comment in enumerate(submission.comments):
        
            try:

                comment_i = comment.body
                comment_dict[i] = comment_i
            except:
                pass

        data['comment_dict'].append(comment_dict)


    submission_df = pd.DataFrame(data)
    submission_df['created_at'] = pd.to_datetime(submission_df['created_at'], unit='s') 
    return submission_df

In [14]:
n_posts = 200 #we recommend lowering these numbers while you build your code.  Start with 1-2!

submission_df = create_submission_df(n_posts=n_posts)


RequestException: error with request HTTPSConnectionPool(host='www.reddit.com', port=443): Max retries exceeded with url: /api/v1/access_token (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1129)')))

In [ ]:
submission_df.head() 

NameError: name 'submission_df' is not defined